In [1]:
import scipy
import numpy as np
from copy import deepcopy
import random
import math
import pandas as pd
import collections
import seaborn as sns

Define Classes:
 - "Variable" allows an id, # of trials, # of successes to be associated with text
 - "VariableGenerator" creates a message bank and can choose messages randomly
   - It will randomly choose from the messages that have the least # of trials

In [2]:
class VariableGenerator:
    pass
        

class Variable: 
    def __init__(self, id:int, content:str="", vartype:str=""):
        """Useful for tracking a message and its success rate"""
        self.vartype = vartype
        self.content = content
        self.id = id
        self.trials = 0
        self.successes = 0
        self.generator =  None
        
    def declareTrial(self)->int:
        self.trials+=1
        self.generator.declareTrial()
        return self.trials
    
    def declareSuccess(self)->int:
        self.successes+=1
        self.generator.declareSuccess()
        return self.successes
        
    def getContent(self)->int:
        return self.content

    def getType(self)->str:
        return self.vartype
    
    def getID(self)->int:
        return self.id

    def getTrials(self)->int:
        return self.trials

    def getSuccesses(self)->int:
        return self.successes

    def fullDescription(self)->dict:
        d = {"varID"     : self.getID(),
             "varType"   : self.getType(),
             "varContent": self.getContent(),
             "varTrials" : self.getTrials(),
             "varSuccesses": self.getSuccesses()}
        return d
        
    def assignGenerator(self,varGen:VariableGenerator)->None:
        self.generator = varGen

    def reportSuccess(self):
        sr = 0 if self.trials==0 else round(self.getSuccesses()/self.getTrials(),2)
        return f"Out of {self.trials}, {self.getContent()[:10]}... has a {sr}% success rate"
        
    def __repr__(self):
        return f"{self.getType()}: {self.getID()}"


class VariableGenerator:
    def __init__(self,vartype:str=""):
        """Useful for generating messages randomly from a list of possible messages"""
        self.Bank=set()
        self.varType=vartype
        self.leastTrials=0
        self.lastID=0
        self.successes=0
        self.trials=0

    def getBank(self):
        return self.Bank

    def generateVariable(self)->str:
        """Get all messages with least trials, return random choice"""
        if not self.leastTrials:
            self.leastTrials = min([m.getTrials() for m in self.Bank])
        minTrials = [m for m in self.Bank if m.getTrials()<=self.leastTrials]

        # If all messages have been sampled more often than minTrials,
        # then randomly sample the message bank.
        if not minTrials:
            randmessage = random.sample(self.Bank,1)[0]
            return randmessage
        randmessage = random.choice(minTrials)
        return randmessage

    def addText(self, text:str)->None:
        """Add message to message bank, update least trials in message bank"""
        var = Variable(self.lastID,text,self.varType)
        self.Bank.add(var)
        var.assignGenerator(self)
        self.lastID += 1
        if self.leastTrials is None:  
            self.leastTrials = 0
    
    def declareSuccess(self)->None:
        self.successes+=1

    def declareTrial(self)->None:
        self.trials+=1

    def dropMessage(self,var:Variable)->None:
        self.Bank.remove(var)

    def getType(self)->str:
        return self.varType

    def exportVariables(self)->pd.DataFrame:
        d = collections.defaultdict(list)
        
        for var in [v.fullDescription() for v in self.getBank()]:
            for key,val in var.items():
                d[key].append(val)

        df = pd.DataFrame.from_dict(d).set_index('varID').sort_values('varID')

        return df
    
    def __repr__(self):
        return f"This {self.vartype} generator has generated {self.trials} variables with {self.successes} successes"


class Experiment:
    def __init__(self, varList:list(), id:int):
        self.id = id
        self.variables = varList
        self.successes = 0
        self.trials = 0

    def declareSuccess(self)->None:
        decSuc = lambda x: x.declareSuccess()
        for var in self.variables: decSuc(var)
        self.successes+=1
    
    def sample(self)->None:
        decTri = lambda x: x.declareTrial()
        for var in self.variables: decTri(var)
        self.trials+=1

    def getID(self)->int:
        return self.id

    def fullDescription(self)->dict:
        descript=dict()
        descript["experimentID"] = self.getID()
        for v in self.variables:
            vartype = v.getType()
            varcontent = v.getID()
            descript[vartype]=varcontent
        descript["TrialCount"]=self.trials
        descript["SuccessCount"]=self.successes
        return descript

    def getVariables(self)->list:
        return self.variables

    def getTrials(self)->int:
        return self.trials

    def __repr__(self):
        getDescrip = lambda x: ":".join([x.getType(),x.getContent()])
        return "\n".join(map(getDescrip,self.variables))


class ExperimentGenerator:
    """ Useful for generating experiments with pre-configured message generators
        Stores
            - ID
            - Experiment Objects
            - Message Generators
        Generates 
            - Experiments, with random messages in them """
    
    def __init__(self, variableGenerators:list()):
        self.variableGenerators = variableGenerators
        self.lastID = 0
        self.experiments = list()

    def generateExperiment(self)->Experiment:
        """ 
            1 Generate a message for every message generator provided
            2 Generate an experiment with an ID and a list of messages
        """
        genVar = lambda x: x.generateVariable()
        varList = list(map(genVar,self.variableGenerators))
        experi = Experiment(varList,self.lastID)
        self.experiments.append(experi)
        self.lastID+=1
        return experi

    def getExperiments(self)->list:
        return self.experiments

    def getGenerators(self)->list:
        return self.variableGenerators

    def exportExperiments(self)->pd.DataFrame:
        d = collections.defaultdict(list)
        for exp in self.getExperiments():
            _ = exp.fullDescription()
            for key, val in _.items():
                d[key].append(val)
        df = pd.DataFrame.from_dict(d).set_index('experimentID')
        return df


class Customer:
    def __init__(self, id:int, traits:list=list()):
        self.id         = id
        self.traits     = traits
        self.experiment = None

    def assignExperiment(self, experiment:Experiment)->None:
        self.experiment = experiment
        self.experiment.sample()

    def setTraits(self, traits:list)->None:
        self.traits = traits
        
    def getTraits(self)->list:
        return self.traits
    
    def getID(self)->int:
        return self.id
    
    def fullDescription(self)->dict:
        d = dict()
        d["custID"]=self.getID()
        d["expID"]=self.experiment.getID() if self.experiment else None
        for t in self.getTraits():
            t_type = t.getType()
            t_id   = t.getID()
            d[t_type]=t_id

        return d
    
    def __repr__(self):
        return f"Customer {self.id}: Experiment {self.experiment}"


class CustomerGenerator:
    def __init__(self, traitGens:list):
        self.customers = list()
        self.traitGens  = traitGens

    def newCustomer(self, id:int)->Customer:
        traits = [t.generateVariable() for t in self.traitGens]
        cust = Customer(id,traits)
        self.customers.append(cust)
        return cust

    def getCustomers(self)->list:
        return self.customers

    def exportCustomers(self)->pd.DataFrame:
        d = collections.defaultdict(list)
        for c in [cust.fullDescription() for cust in self.getCustomers()]:
            for key,val in c.items():
                d[key].append(val)
        df = pd.DataFrame.from_dict(d).set_index('custID')
        return df



Generate Variables

In [3]:
introductionMessages=["hi","hello","epa"]
pitchMessages=["give me $5 because I'm nice", "give me $5 for a burger", "give me $5 for an EKG"]
qualifyMessages=["are you 18+?", "are you a doctor?", "are you american?"]

message1 = VariableGenerator("Message 1") # Introduction
message2 = VariableGenerator("Message 2") # Qualification
message3 = VariableGenerator("Message 3") # Pitch

for i in introductionMessages: message1.addText(i)
for i in qualifyMessages: message2.addText(i)
for i in pitchMessages: message3.addText(i)

gens = [message1,message2,message3]

Generate Experiments

In [4]:
expGen = ExperimentGenerator(gens)

Export Variable Generators with message IDs.

Locally store experiment generator, variable generator, and last ID for both.
 so that there can be a continuity with the experiment IDs.


In [5]:
n_cust = 1200
profileNames = ["RN", "Office Mgr", "Internal Med"]
funnelNames = ["No Response","Response","Demo","Purchase"]

profiles = VariableGenerator("Profile")
funnels = VariableGenerator("Funnel Stage")

for i in profileNames: profiles.addText(i)
for i in funnelNames: funnels.addText(i)

traits = [profiles,funnels]

custGen = CustomerGenerator(traits)
customerIDs = set(range(n_cust))
custPopulation = {custGen.newCustomer(ID) for ID in customerIDs}

In [6]:
trials = 50
experiments = math.ceil(n_cust/trials)

for i in range(experiments):
    exp = expGen.generateExperiment()
    custSample = random.sample(custPopulation,50)
    custPopulation = custPopulation.difference({c for c in custSample})
    for cust in custSample: cust.assignExperiment(exp)

/var/folders/2w/1s6grqld6pv4xtj9g5qxl4tm0000gn/T/ipykernel_5101/254490944.py:6: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  custSample = random.sample(custPopulation,50)
/var/folders/2w/1s6grqld6pv4xtj9g5qxl4tm0000gn/T/ipykernel_5101/1960901473.py:81: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  randmessage = random.sample(self.Bank,1)[0]


In [7]:
descVar = lambda x: x.exportVariables()
list(map(descVar,gens))

[         varType varContent  varTrials  varSuccesses
 varID                                               
 0      Message 1         hi        600             0
 1      Message 1      hello        250             0
 2      Message 1        epa        350             0,
          varType         varContent  varTrials  varSuccesses
 varID                                                       
 0      Message 2       are you 18+?        350             0
 1      Message 2  are you a doctor?        550             0
 2      Message 2  are you american?        300             0,
          varType                   varContent  varTrials  varSuccesses
 varID                                                                 
 0      Message 3  give me $5 because I'm nice        250             0
 1      Message 3      give me $5 for a burger        550             0
 2      Message 3        give me $5 for an EKG        400             0]

In [8]:
expGen.exportExperiments()

,Message 1,Message 2,Message 3,TrialCount,SuccessCount
experimentID,,,,,
0,2,1,1,50,0
1,0,0,2,50,0
2,1,2,0,50,0
3,2,0,2,50,0
4,1,2,0,50,0
5,0,1,1,50,0
6,2,2,2,50,0
7,0,1,2,50,0
8,0,1,2,50,0


In [9]:
list(map(descVar,traits))

[       varType    varContent  varTrials  varSuccesses
 varID                                                
 0      Profile            RN          0             0
 1      Profile    Office Mgr          0             0
 2      Profile  Internal Med          0             0,
             varType   varContent  varTrials  varSuccesses
 varID                                                    
 0      Funnel Stage  No Response          0             0
 1      Funnel Stage     Response          0             0
 2      Funnel Stage         Demo          0             0
 3      Funnel Stage     Purchase          0             0]

In [10]:
custGen.exportCustomers()

,expID,Profile,Funnel Stage
custID,,,
0,5,0,2
1,19,1,0
2,11,0,0
3,18,2,3
4,11,0,1
...,...,...,...
1195,6,1,3
1196,18,1,1
1197,20,0,2


Load 1200 Target Profiles

Assign Experimental Variables to Target Profiles

Upload to Linked Helper as Custom Variables for a Campaign

Include Experiment ID and Details in WebHook to Zoho

References


LinkedHelper Custom Variables Reference
https://www.youtube.com/watch?v=NfYsxPjtji8&t=5s

LinkedIn WebHook to Zapier (with custom variables)
https://support.linkedhelper.com/hc/en-us/articles/360020407940-How-to-send-profiles-to-Google-Sheets-or-any-other-CRM-webapp-via-Zapier-Webhook#h_01F85MMEXVD7AF3QT0XQQ20GCE

Receiving Webhooks and Updating Customers
https://www.youtube.com/watch?v=fb7hqGAx8Ek